In [2]:
import mysql.connector
from mysql.connector import errorcode
import datetime

config={
     'user':'user1',
     'password':'user1',
     'host':'localhost',
     'port':"3308",
     'database':'python_expenses_db'
}
cn=mysql.connector.connect(**config,buffered=True)
cursor = cn.cursor(buffered=True)

while(True):
    print("\n\n=====================================================")
    print(" -=-=-| Welcome to the Expense Tracker system |-=-=-   ")
    print("=====================================================")
    print("Enter 1. To Login")
    print("Enter 2. To Register")
    print("Enter 3. To Exit")
    print("=====================================================")
    choice_login=int(input("Enter choice:"))
    if choice_login==1:
        cursor.execute("SELECT * FROM `register` WHERE username=%s AND password=%s",(input("Enter username:"),input("Enter password:")))
        temp=(cursor.fetchone())
        if temp is not None:
            userid=temp[0]
            while(True):
                print("\n\nOPERATIONS")
                print("\n=====================================================")
                print("Enter 1. To Add Income")
                print("Enter 2. To Manage Expenses")
                print("Enter 3. To Logout")
                print("=====================================================")
                choice_income=int(input("Enter choice:"))
                if choice_income==1:
                    sql_qry="SELECT user_id FROM `income_tbl`"
                    cursor.execute(sql_qry)
                    flag=0
                    for uid in cursor:
                        if uid[0]==userid:
                            sql_qry="UPDATE `income_tbl` SET income=%s WHERE user_id=%s"
                            cursor.execute(sql_qry,(float(input("Add Income:")),userid))
                            cn.commit()
                            print('Income updated successfully.')
                            flag=1
                            break
                    if flag==0:
                        sql_qry="INSERT INTO `income_tbl`(income,user_id) values(%s,%s)"
                        cursor.execute(sql_qry,(float(input("Add Income:")),userid))
                        cn.commit()
                        print('Income added successfully....')
                        sql_qry="INSERT INTO `budget`(budget,user_id) values(%s,%s)"
                        cursor.execute(sql_qry,(0,userid))
                        cn.commit()
                    
                elif choice_income==2:
                    while(True):
                        print("\n\nMANAGE OPTIONS:")
                        print("\n=====================================================")
                        print("Enter 1. To Manage Income")
                        print("Enter 2. To Manage Expenses")
                        print("Enter 3. To Manage Budget")
                        print("Enter 4. To Logout")
                        print("=====================================================")
                        choice_expense=int(input("Enter choice:"))
                        if choice_expense==1:
                            while(True):
                                print("\n\nMANAGE INCOME:")
                                print("\n=====================================================")
                                print("Enter 1. To Update Income")
                                print("Enter 2. To Show Income")
                                print("Enter 3. To Exit")
                                print("=====================================================")
                                choice_1=int(input("Enter choice:"))
                                
                                if choice_1==1:
                                    sql_qry="SELECT income FROM `income_tbl` WHERE user_id=%s"
                                    cursor.execute(sql_qry,(userid,))
                                    cur_income=cursor.fetchone()[0]
                                    cur_income+=float(input("Updated Income:"))
                                    sql_qry="UPDATE `income_tbl` SET income=%s WHERE user_id=%s"
                                    cursor.execute(sql_qry,(cur_income,userid))
                                    cn.commit()
                                    print('Income updated successfully.')
                                    
                                elif choice_1==2:
                                    sql_qry="SELECT * FROM `income_tbl` WHERE user_id=%s"
                                    cursor.execute(sql_qry,(userid,))
                                    temp=cursor.fetchone()
                                    print("Current Income : ",temp[1])
                                    
                                elif choice_1==3:
                                    break
                                else:
                                    print("enter valid choice")
                                    
                                    
                        elif choice_expense==2:
                            while(True):
                                print("\n\nMANAGE EXPENSES:")
                                print("\n=====================================================")
                                print("Enter 1. To Add Expense")
                                print("Enter 2. To Remove Expense")
                                print("Enter 3. To View Expenses")
                                print("Enter 4. To Exit")
                                print("=====================================================")
                                choice_exp=int(input("Enter choice:"))
                                
                                if choice_exp==1:
                                    sql_select_qry="SELECT id,income FROM `income_tbl` WHERE user_id=%s"
                                    cursor.execute(sql_select_qry,(userid,))
                                    t=cursor.fetchone()
                                    incomeid=t[0]
                                    curr_income=t[1]
                                    
                                    expense_name=input("Add Expense:")
                                    expense_price=int(input("Price for above expense:"))
                                    
                                    sql_qry="SELECT budget FROM `budget` WHERE user_id=%s"
                                    cursor.execute(sql_qry,(userid,))
                                    curr_budget=cursor.fetchone()[0]
                                    
                                    if curr_budget>expense_price:
                                        
                                        curr_budget-=expense_price
                                        sql_qry="UPDATE `budget` SET budget=%s WHERE user_id=%s"
                                        cursor.execute(sql_qry,(curr_budget,userid))
                                        cn.commit()
                                        curr_income-=expense_price
                                        sql_qry="UPDATE `income_tbl` SET income=%s WHERE user_id=%s"
                                        cursor.execute(sql_qry,(curr_income,userid))
                                        cn.commit()
                                        print('expense deducted from income and budget...')
                                    
                                    else:
                                        print("Your expense is exceeding your current budget..... Do you want to add this expense ?")
                                        ch=input("[y/n] ? :")
                                        if ch=='y':
                                            if curr_income<expense_price:
                                                print("You cannot do this expense... your income is too low.")
                                            else:
                                                curr_income-=expense_price
                                                sql_qry="UPDATE `income_tbl` SET income=%s WHERE user_id=%s"
                                                cursor.execute(sql_qry,(curr_income,userid))
                                                cn.commit()
                                                sql_qry="UPDATE `budget` SET budget=%s WHERE user_id=%s"
                                                cursor.execute(sql_qry,(0,userid))
                                                cn.commit()
                                        else :
                                            pass
        
                                    print("Current Budget is :" ,curr_budget)
                                    sql_qry="INSERT INTO `expenses`(expense,price,user_id,income_id) values(%s,%s,%s,%s)"
                                    cursor.execute(sql_qry,(expense_name,expense_price,userid,incomeid))
                                    cn.commit()
                                    print('Data entered successfully.')
                                    
                                    sql_select_qry="SELECT price FROM `expenses` WHERE user_id=%s"
                                    cursor.execute(sql_select_qry,(userid,))
                                    total_expenses=0
                                    for pr in cursor:
                                         total_expenses+=pr[0]
                                    print(total_expenses)
                                    
                                elif choice_exp==2:
                                    sql_select_qry="SELECT id,expense,price FROM `expenses` WHERE user_id=%s"
                                    cursor.execute(sql_select_qry,(userid,))
                                    print("id\t expense\t price")
                                    
                                    for exp in cursor:
                                        print(exp[0],"\t",exp[1],"\t",exp[2])
                                    sql_delete_qry="DELETE FROM expenses WHERE id=%s"
                                    temp=(cursor.execute(sql_delete_qry,(int(input("Enter id for deletion:")),)))
                                    cn.commit()
                                    
                                    if(not temp):
                                        print("Expense removed..")
                                    else:
                                        print("Expense not removed")
                                    
                                elif choice_exp==3:
                                    sql_select_qry="SELECT expense,price FROM `expenses` WHERE user_id=%s"
                                    cursor.execute(sql_select_qry,(userid,))
                                    print("expense\t\t price")
                                    for exp in cursor:
                                        print(exp[0],"\t",exp[1])
                                    
                                elif choice_exp==4:
                                    break
                                    
                                else:
                                    print("enter valid choice")
                                
                        elif choice_expense==3:
                            while(True):
                                print("\n\nMANAGE BUDGET:")
                                print("\n\n=====================================================")
                                print("Enter 1. To Add Budget")
                                print("Enter 2. To Update Budget")
                                print("Enter 3. To View Budget")
                                print("Enter 4. To Logout")
                                print("=====================================================")
                                choice_budget=int(input("Enter choice:"))
                                if choice_budget==1:
                                    sql_qry="SELECT user_id FROM `budget`"
                                    cursor.execute(sql_qry)
                                    flag=0
                                    for uid in cursor:
                                        if uid[0]==userid:
                                            sql_qry="UPDATE `budget` SET budget=%s WHERE user_id=%s"
                                            cursor.execute(sql_qry,(float(input("Add budget:")),userid))
                                            cn.commit()
                                            print('budget updated successfully.')
                                            flag=1
                                            break
                                    if flag==0:
                                        budget_amt=int(input("Add Budget : "))
                                        sql_qry="SELECT income FROM `income_tbl` WHERE user_id=%s"
                                        cursor.execute(sql_qry,(userid,))
                                        curr_income=cursor.fetchone()[0]
                                        if curr_income < curr_budget:
                                            print("Set less budget than your current income..current income is :",curr_income)

                                        else:
                                            sql_qry="INSERT INTO `budget`(budget,user_id) values(%s,%s)"
                                            cursor.execute(sql_qry,(budget_amt,userid))
                                            cn.commit()
                                            print('budget entered successfully.')

                                elif choice_budget==2:
                                    budget_amt=int(input("Add new Budget : "))
                                    sql_qry="SELECT income FROM `income_tbl` WHERE user_id=%s"
                                    cursor.execute(sql_qry,(userid,))
                                    curr_income=cursor.fetchone()[0]
                                    if curr_income < curr_budget:
                                        print("Set less budget than your current income..current income is :",curr_income)
                                    
                                    else:
                                        sql_qry="UPDATE `budget` SET budget=%s WHERE user_id=%s"
                                        cursor.execute(sql_qry,(budget_amt,userid))
                                        cn.commit()
                                        print('budget updated successfully.')
                                        
                                elif choice_budget==3:
                                    sql_qry="SELECT budget FROM `budget` WHERE user_id=%s"
                                    cursor.execute(sql_qry,(userid,))
                                    print("Current Budget is :" ,(cursor.fetchone())[0])
                                    
                                elif choice_budget==4:
                                    break;
                                else:
                                    print("enter valid choice")
                        
                        elif choice_expense==4:
                            break
                        else:
                            print("enter valid choice")
                            
                elif choice_income==3:
                    break
                else:
                    print("enter valid choice")
        else:
            print("Invalid username and password..")
    elif choice_login==2:
        sql_qry="INSERT INTO `register`(username,password,name) VALUES(%s,%s,%s)"
        cursor.execute(sql_qry,(input("Enter username:"),input("Enter password:"),input("Enter name:")))
        cn.commit()
        print('Data entered successfully.')
    elif choice_login==3:
        break
    else:
        print("enter valid choice")

cursor.close()
cn.close()





 -=-=-| Welcome to the Expense Tracker system |-=-=-   
Enter 1. To Login
Enter 2. To Register
Enter 3. To Exit
Enter choice:1
Enter username:niki
Enter password:niki
Invalid username and password..


 -=-=-| Welcome to the Expense Tracker system |-=-=-   
Enter 1. To Login
Enter 2. To Register
Enter 3. To Exit
Enter choice:1
Enter username:niki
Enter password:niki123


OPERATIONS

Enter 1. To Add Income
Enter 2. To Manage Expenses
Enter 3. To Logout
Enter choice:1
Add Income:40000
Income updated successfully.


OPERATIONS

Enter 1. To Add Income
Enter 2. To Manage Expenses
Enter 3. To Logout
Enter choice:2


MANAGE OPTIONS:

Enter 1. To Manage Income
Enter 2. To Manage Expenses
Enter 3. To Manage Budget
Enter 4. To Logout
Enter choice:1


MANAGE INCOME:

Enter 1. To Update Income
Enter 2. To Show Income
Enter 3. To Exit
Enter choice:3


MANAGE OPTIONS:

Enter 1. To Manage Income
Enter 2. To Manage Expenses
Enter 3. To Manage Budget
Enter 4. To Logout
Enter choice:4


OPERATIONS

Ent